In [2]:
#MODULES 
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import cv2 
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

print(tf.__version__)

2.2.0


In [3]:
# SETTING DIRECTORIES AND CONSTANT VALUES 
TRAIN_DIR = 'C:\\Users\\Hp\\Desktop\\Wobot\\Kaggle1\\dogs-vs-cats\\train\\train'
TEST_DIR = 'C:\\Users\\Hp\\Desktop\\Wobot\\Kaggle1\\dogs-vs-cats\\test1\\test1'
IMAGE_SIZE = 50
ALPHA = 0.001 # Hyperparameter: Learning Rate alpha 

In [4]:
MODEL_NAME = 'dvc'

In [72]:
#ONE HOT ENCODING
# Cats [1,0]
# Dogs [0,1]

def img_label(img):
    label = img.split('.')[-3]
    if label == 'cat':
        return np.array([1,0])
    elif label == 'dog':
        return np.array([0,1])
    else:
        print("Invalid image ")
        return np.array([0,0])
    
def rev_label(arr):
    if np.array_equal(arr, np.array([1,0])):
        return 1
    elif np.array_equal(arr, np.array([0,1])):
        return 0
    else:
        print("wtf just happened bruh")

In [89]:
# HELPER FUNCTION TO SEE DATA 
def print_array_image(index, dataType = 'train'):
    np.set_printoptions(linewidth=200)
    if dataType == 'train':
        plt.imshow(train_data[index])
        print('Label :',train_labels[index])
        print('Numpy array:',train_data[index])
    elif dataType == 'test':
        plt.imshow(test_data[index])
        print(test_id[index])
        print(test_data[index])
    else:
        print("Enter right type of data")

In [7]:
#CREATE TRAINING DATA 
def create_training_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = img_label(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation = cv2.INTER_NEAREST)
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    return np.array(training_data)

In [8]:
# INITIALISE TRAINING DATA
training_data = create_training_data()
train_data = np.array([i[0] for i in training_data])
train_labels = np.array([i[1] for i in training_data])

100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [00:35<00:00, 698.18it/s]


In [74]:
print(train_data.shape) # (25000,50,50)
print(train_labels.shape) # (2500,2)
print(len(train_labels))

(25000, 50, 50)
(25000, 2)
25000


In [52]:
# PROCESSING TEST DATA 
def process_testing_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        img_id = img.split('.')[-2]
        path = os.path.join(TEST_DIR, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation = cv2.INTER_NEAREST)
        testing_data.append([np.array(img), img_id])
    return np.array(testing_data)

In [53]:
testing_data = process_testing_data()
test_data = np.array([i[0] for i in testing_data])
test_id = np.array([i[1] for i in testing_data])

100%|███████████████████████████████████████████████████████████████████████████| 12500/12500 [00:16<00:00, 766.05it/s]


In [55]:
print(testing_data.shape) # (12500, 2)
print(test_data.shape) # (12500, 50, 50)

(12500, 2)
(12500, 50, 50)


In [92]:
# PRE PROCESSING STEP 
train_data  = train_data / 255.0
test_data = test_data / 255.0

In [94]:
train_class = np.zeros((len(train_labels)))
print(train_class.shape)
for i in range(len(train_labels)):
    train_class[i] = rev_label(train_labels[i])

(25000,)


In [97]:
# CROSS VALIDATION 
traind = np.expand_dims(train_data[:20000] , axis=-1) 
trainc = np.expand_dims(train_class[:20000], axis = -1)

testd = np.expand_dims(train_data[20000:] , axis=-1)
testc = np.expand_dims(train_class[20000:], axis = -1)

In [98]:
print(traind.shape)
print(trainc.shape)
print(testd.shape)
print(testc.shape)

(20000, 50, 50, 1)
(20000, 1)
(5000, 50, 50, 1)
(5000, 1)


In [99]:
from tensorflow.keras import datasets, layers, models

In [109]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50,50,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2))

In [110]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 9, 9, 64)          36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)             

In [112]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(traind, trainc, epochs=15,
                    validation_data=(testd, testc))

Epoch 1/15
625/625 [==============================] - 54s 87ms/step - loss: 0.3915 - accuracy: 0.8319 - val_loss: 0.4442 - val_accuracy: 0.7952
Epoch 2/15
625/625 [==============================] - 55s 88ms/step - loss: 0.3692 - accuracy: 0.8393 - val_loss: 0.4095 - val_accuracy: 0.8142
Epoch 3/15
625/625 [==============================] - 53s 84ms/step - loss: 0.3411 - accuracy: 0.8530 - val_loss: 0.4407 - val_accuracy: 0.8016
Epoch 4/15
625/625 [==============================] - 52s 83ms/step - loss: 0.3191 - accuracy: 0.8623 - val_loss: 0.4920 - val_accuracy: 0.7846
Epoch 5/15
625/625 [==============================] - 53s 85ms/step - loss: 0.2922 - accuracy: 0.8784 - val_loss: 0.4226 - val_accuracy: 0.8126
Epoch 6/15
625/625 [==============================] - 55s 88ms/step - loss: 0.2696 - accuracy: 0.8876 - val_loss: 0.4523 - val_accuracy: 0.8082
Epoch 7/15
625/625 [==============================] - 52s 83ms/step - loss: 0.2484 - accuracy: 0.8980 - val_loss: 0.4469 - val_accuracy:

In [ ]:
with open('submission')